[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Imaging-AI-for-Health-virtual-lab/SHAP-in-repeated-nested-CV/blob/main/regression_ICBM.ipynb)

# REPEATED NESTED CROSS-VALIDATION FOR Multilayer perceptron - Testing on glucose DATASET

Cloning GitHub repository:

Install dependencies and import modules (Unchanged)

In [ ]:
####### Import packages ########################
import numpy as np
np.random.seed(42)
np.seterr(divide='ignore', invalid='ignore')
import pandas as pd
from sklearn.model_selection import KFold, RandomizedSearchCV, cross_validate
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import pickle, keras
import shap
import tensorflow as tf
import sklearn
from keras.wrappers.scikit_learn import KerasRegressor
import os

Define functions

In [ ]:

def average_shap_values(dir_name, data, num_trials, num_splits):
    """
    Takes shap values from .pkl files computed for different folds and trials and makes a mathematical average, and plots the final results.
    --------------------------
    Parameters:

    - dir_name : <string>, path in which there is the directory "Shap_values/"
    - data : <pandas dataframe>, data used to compute SHAP values
    - num_trials : <int>, number of repetitions
    - num_splits: <int>, number of splits for the nested cross-validation
    ------------------------

    Returns:
    - shaps: <list>, list of two pandas Dataframes containing the SHAP values for training set and test set separately.

    """
    shaps = []
    for name in ["train", "test"]:
        final_shap_values = None
        for num_trial in range(num_trials):
            for num_split in range(num_splits):
                file = open(dir_name + "Shap_values/" + name + str(num_trial) + "fold" + str(num_split) + ".pkl", "rb")
                shap_values = pickle.load(file)
                if final_shap_values is None:
                    final_shap_values = shap_values
                else:
                    final_shap_values = final_shap_values.append(shap_values)

        features = list(shap_values.columns)
        features.remove("index")

        df = pd.DataFrame(columns=features)
        for i in range(len(data)):
            filt = final_shap_values['index'] == i

            df = df.append(final_shap_values[filt].sum(axis=0, numeric_only=True), ignore_index=True)
        del df["index"]

        if "train" in name:
            df = df / num_trials / (num_splits - 1)
        else:
            df = df / num_trials

        shaps.append(df)

    return shaps

def kernel_constraint_func(int):
    if int == -1:
        return None
    elif int == 2:
        return tf.keras.constraints.max_norm(2)
    elif int == 3:
        return tf.keras.constraints.max_norm(3)
    elif int == 4:
        return tf.keras.constraints.max_norm(4)

## Setting variables

In [ ]:

####### SETTING VARIABLES ########################
experiment_name = "XAI"
dir_name = "nested_cv_keras/"
os.makedirs(dir_name, exist_ok=True)
os.makedirs(dir_name + "Shap_values", exist_ok=True)
os.makedirs(dir_name + "plots", exist_ok=True)
os.makedirs(dir_name + "average_plots", exist_ok=True)

num_trials = 10 # Number of repetitions (Decreased to 10, about 16 hours computation time)
num_splits = 5 #Number of folds (Default)
n_iter= 1000 #Number of iteration for hyperparameter tuning (We increased to 1000)

## Reading and filtering data

Input data (Glucose condition)

In [ ]:
X_data_raw  = pd.read_feather("data/simulated_fluxes(glc).feather").set_index("index")

#Remove any unnecessary columns(reactions)
X_data  = pd.DataFrame(index= X_data_raw.index)
for index_col in X_data_raw.columns:
    each_column = X_data_raw.loc[:,index_col]
    is_neg = 0
    is_pos = 0
    for f_value in each_column:
        if f_value <0 :
            is_neg =1
        if f_value > 0:
            is_pos =1

    if  is_neg ==0 and is_pos==0:
        continue

    else:
        X_data[each_column.name] = abs(each_column)

Output data (Glucose condition)

In [ ]:
#Extracting growth data for target data
growth_data = pd.read_feather("data/biomass_data.feather").set_index("index")
y_data_raw =  growth_data["glc"]
y_data = y_data_raw[y_data_raw.index.isin(X_data.index)]

features = list(X_data.columns)

## Defining variables and metrices for nested cross validation

In [ ]:
train_scores = np.zeros(num_trials)
test_scores = np.zeros(num_trials)

mse_train_scores = np.zeros(num_trials)
r2_train_scores = np.zeros(num_trials)
neg_mean_absolute_error_train_scores = np.zeros(num_trials)
correlation_train_scores = np.zeros(num_trials)

mse_test_scores = np.zeros(num_trials)
r2_test_scores = np.zeros(num_trials)
neg_mean_absolute_error_test_scores = np.zeros(num_trials)
correlation_test_scores = np.zeros(num_trials)


def my_custom_loss_func(y_true, y_pred):
    return np.corrcoef(y_true,y_pred)[0][1]

score = make_scorer(my_custom_loss_func)

myscoring = {'mse': 'neg_mean_squared_error',
        'r2': 'r2',
        'neg_mean_absolute_error':'neg_mean_absolute_error',
        'correlation' : score
        }

## Model setting

In [ ]:
#We reduced the size of hyperparameter space, but made sure to include parameter that were selected in the previous deep learning tuning.

######## Hyperparamter SETTING ##############################
layers = [0,1,2,3]
neurons           = [10, 25, 50, 100, 200, 1000] # number of perceptrons for each layers
optimizer_param   = ['adam', 'rmsprop', 'sgd'] # backpropagation optimizers
learning_rate     = [0.1, 0.01,0.005,0.001]
kernel_constraint = [-1,2,3] # layer weight constraints, -1 : no constraint
dropout           = [0.4, 0.5, 0.6] # Dropout layer rate


######## MODEL SETTING ##############################
def build_model(layers, neurons,optimizer_param,learning_rate,kernel_constraint,dropout ):
    # Model construction
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(len(X_data.columns),)))
    for i in range(0,layers+1):
        model.add(tf.keras.layers.Dense(units= neurons, activation='relu',
                                        kernel_constraint=kernel_constraint_func(
                                            kernel_constraint)))
        model.add(tf.keras.layers.Dropout(dropout))
    model.add(tf.keras.layers.Dense(1, activation='linear'))

    # Optimizer
    optimizer = optimizer_param

    if optimizer == "adam":
        final_optimizer = tf.optimizers.Adam(learning_rate)
    elif optimizer == "sgd":
        final_optimizer = tf.optimizers.SGD(learning_rate)
    elif optimizer == "rmsprop":
        final_optimizer = tf.optimizers.RMSprop(learning_rate)

    # Compile model
    model.compile(
        optimizer=final_optimizer,
        loss='mse',
        metrics=['mse']
    )
    return model

temp_model = KerasRegressor(build_fn=build_model)

params =  dict(layers = layers,
               neurons =neurons,
               optimizer_param = optimizer_param,
               learning_rate=learning_rate,
               kernel_constraint=kernel_constraint,
               dropout=dropout
               )
#####################################################

## Start of repeated nested CV

In [ ]:
X = sklearn.preprocessing.StandardScaler().fit_transform(X_data)
X = pd.DataFrame(X, columns=features)
y = y_data


shap.initjs()
for i in range(num_trials):

    print("Iteration:" + str(i))

    inner_cv = KFold(n_splits=num_splits, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=num_splits, shuffle=True, random_state=i)

    
    clf = RandomizedSearchCV(estimator=temp_model, param_distributions=params, n_iter=n_iter, scoring=score,
                            refit=True, cv=inner_cv, verbose=0,  random_state=i, return_train_score=True,
                            n_jobs =1) 


    nested_score = cross_validate(clf, X=X, y=y, cv=outer_cv, return_train_score=True,return_estimator=True, scoring = myscoring)


    mse_test_scores[i] = np.mean(nested_score['test_mse'])
    r2_test_scores[i] = np.mean(nested_score['test_r2'])
    neg_mean_absolute_error_test_scores[i] = np.mean(nested_score['test_neg_mean_absolute_error'])
    correlation_test_scores[i] = np.mean(nested_score['test_correlation'])

    print('mse ' + str(mse_test_scores[i]))
    print('r2 ' + str(r2_test_scores[i]))
    print('neg_mean_absolute_error ' + str(neg_mean_absolute_error_test_scores[i]))
    print('correlation ' + str(correlation_test_scores[i]))


    ############# SHAP VALUES COMPUTATION FOR EACH FOLD ##################
    iter_shap = 0
    indices = []

    for train_index, test_index in inner_cv.split(X, y):
        #print("Split:", iter_shap)
        ## TRUE POSITIVE RATE COMPUTATION FOR EACH OUTER LOOP (TEST SET)
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        X_train = pd.DataFrame(X_train,columns=features)
        X_test = pd.DataFrame(X_test,columns=features)
        y_train = pd.DataFrame(y_train)


        regressor_model = nested_score['estimator'][iter_shap].best_estimator_.model
        print(regressor_model.summary())

        X_train = np.array(X_train)
        X_test = np.array(X_test)

        regressor_model.fit(X_train, y_train, epochs =40, verbose=0)
        #y_pred = regressor_model.predict(X_test)

        #train_tmp_df = pd.DataFrame(X_train, columns=features)
        explainer = shap.DeepExplainer(regressor_model, X_train)
        df_shap = pd.DataFrame(explainer.shap_values(X_train,check_additivity=False)[0], columns=features)
        df_shap["index"] = train_index
        pickle.dump(df_shap,
                    open(dir_name + '/Shap_values/' + 'train' + str(i) + 'fold' + str(iter_shap) + '.pkl', 'wb'))


        #test shap
        test_explainer = shap.DeepExplainer(regressor_model,X_test)
        test_df_shap = pd.DataFrame(explainer.shap_values(X_test,check_additivity=False)[0], columns=features)
        test_df_shap["index"] = test_index
        pickle.dump(test_df_shap,
                    open(dir_name + '/Shap_values/' + 'test' + str(i) + 'fold' + str(iter_shap) + '.pkl', 'wb'))


        iter_shap += 1

# AVERAGE SHAP VALUES FOR TRAIN AND TEST

In [ ]:
averaged_shaps = average_shap_values(dir_name,X_data_raw[features], num_trials,num_splits)

## TRAIN SHAP VALUES

In [ ]:
training_shap=averaged_shaps[0]
training_shap_df_median = pd.DataFrame(training_shap.median())
training_shap_df_median.columns = ["SHAP value"]
training_shap_df_median = training_shap_df_median.sort_values(["SHAP value"])

#Extract each reaction's SHAP value
raw_SHAP_values = training_shap_df_median

#Filter out transport and external reactions
memote_pure_rxn = open("util/memote_pure_rxns.txt", 'r').read().strip('"').split('","')

#Separate beneficial(+) and detrimental(-) reactions based on SHAP value
SHAP_pos = raw_SHAP_values[raw_SHAP_values.iloc[:, 0] > 0]
SHAP_neg = raw_SHAP_values[raw_SHAP_values.iloc[:, 0] < 0]

#Filter out reactions with negligible SHAP value
avg_coefs_pos = SHAP_pos.iloc[:, 0].mean()
avg_coefs_neg = SHAP_neg.iloc[:, 0].mean()

final_pos_SHAPs = SHAP_pos[SHAP_pos.iloc[:,0] >=  0.1*avg_coefs_pos]
final_pos_SHAPs = final_pos_SHAPs[final_pos_SHAPs.index.isin(memote_pure_rxn) == True]
final_neg_SHAPs = SHAP_neg[abs(SHAP_neg.iloc[:,0]) >= abs(0.1*avg_coefs_neg)]
final_neg_SHAPs = final_neg_SHAPs[final_neg_SHAPs.index.isin(memote_pure_rxn) == True]

#Sort and extract to csv
filtered_SHAPs = final_pos_SHAPs.append(final_neg_SHAPs)
filtered_SHAPs = filtered_SHAPs.sort_values(ascending=False, by=["SHAP value"])
filtered_SHAPs.to_csv("output/training_shap.csv")

## TEST SHAP VALUES

In [ ]:
testing_shap = averaged_shaps[1]
testing_shap_median = pd.DataFrame(testing_shap.median())
testing_shap_median.columns = ["SHAP value"]
testing_shap_median = testing_shap_median.sort_values(["SHAP value"])

#Extract each reaction's SHAP value
raw_SHAP_values = testing_shap_median

#Filter out transport and external reactions
memote_pure_rxn = open("util/memote_pure_rxns.txt", 'r').read().strip('"').split('","')

#Separate beneficial(+) and detrimental(-) reactions based on SHAP value
SHAP_pos = raw_SHAP_values[raw_SHAP_values.iloc[:, 0] > 0]
SHAP_neg = raw_SHAP_values[raw_SHAP_values.iloc[:, 0] < 0]

#Filter out reactions with negligible SHAP value
avg_coefs_pos = SHAP_pos.iloc[:, 0].mean()
avg_coefs_neg = SHAP_neg.iloc[:, 0].mean()

final_pos_SHAPs = SHAP_pos[SHAP_pos.iloc[:,0] >=  0.1*avg_coefs_pos]
final_pos_SHAPs = final_pos_SHAPs[final_pos_SHAPs.index.isin(memote_pure_rxn) == True]
final_neg_SHAPs = SHAP_neg[abs(SHAP_neg.iloc[:,0]) >= abs(0.1*avg_coefs_neg)]
final_neg_SHAPs = final_neg_SHAPs[final_neg_SHAPs.index.isin(memote_pure_rxn) == True]

#Sort and extract to csv
filtered_SHAPs = final_pos_SHAPs.append(final_neg_SHAPs)
filtered_SHAPs = filtered_SHAPs.sort_values(ascending=False, by=["SHAP value"])
filtered_SHAPs.columns = ["SHAP value"] 
filtered_SHAPs.to_csv("output/test_shap.csv")